In 2017, Google published eager execution in TensorFlow, which greatly helped the visualization during training. With eager execution, we are able to print out variables and other models without running a session as in the traditional TensorFlow program. And this would be very benificial for new learners. In this notebook, we will use eager execution to learn the basics of TensorFlow such as the data type, GPU assignment and other APIs such as dataset and automatic differentiation.

In [1]:
# Import TensorFlow and activate eager execution.
import tensorflow as tf

tf.enable_eager_execution()

In [2]:
# Tensors calculation and transformation. Python types are converted automatically to Tensor types.
print('tf.add(12, 37) =', tf.add(12, 37))
print('tf.add([1, 3], [4, 8]) =', tf.add([1, 3], [4, 8]))
print('tf.reduce_sum([-4, 0, 8, 10]) =', tf.reduce_sum([-4, 0, 8, 10]))
print()

x = tf.matmul([[1, 3]], [[4, 6], [7, 9]])  # tensors have two properties- shape and dtype
print('x =', x)
print('x.shape:', x.shape)
print('x.dtype:', x.dtype)

tf.add(12, 37) = tf.Tensor(49, shape=(), dtype=int32)
tf.add([1, 3], [4, 8]) = tf.Tensor([ 5 11], shape=(2,), dtype=int32)
tf.reduce_sum([-4, 0, 8, 10]) = tf.Tensor(14, shape=(), dtype=int32)

x = tf.Tensor([[25 33]], shape=(1, 2), dtype=int32)
x.shape: (1, 2)
x.dtype: <dtype: 'int32'>


In [3]:
# Numpy compatibility. TensorFlow tensors and Numpy arrays can be transofrmed easily in type and automatically in calculation.
import numpy as np

numpy_arr = np.ones((1, 3))
print('numpy array:', numpy_arr)
tensor = tf.multiply(numpy_arr, 3)
print('tensor:', tensor)
numpy_sum = np.add(tensor, 1)
print('numpy sum:', numpy_sum)
numpy_tensor = tensor.numpy()
print('numpy tensor:', numpy_tensor)

numpy array: [[1. 1. 1.]]
tensor: tf.Tensor([[3. 3. 3.]], shape=(1, 3), dtype=float64)
numpy sum: [[4. 4. 4.]]
numpy tensor: [[3. 3. 3.]]


In [4]:
# Check if GPU is avaliable and if calculation or variable is a certain GPU.
print(tf.test.is_gpu_available())
print(x.device.endswith('GPU:0'))

False
False


In [5]:
# CPU or GPU assignment.
import time

def time_matmul(x):
    start = time.time()
    for loop in range(10):
        tf.matmul(x, x)
    return (time.time() - start) / 10

with tf.device('CPU:0'):
    x = tf.random_uniform([1000, 1000])
    assert x.device.endswith('CPU:0')
    print('CPU matrix multiplication running time:', time_matmul(x), 's')
    
if tf.test.is_gpu_available():
    with tf.device('GPU:0'):
        x = tf.random_uniform([1000, 1000])
        assert x.device.endswith('GPU:0')
        print('GPU matrix multiplication running time:', time_matmul(x), 's')

CPU matrix multiplication running time: 0.016754460334777833 s


In [6]:
# Dataset API.
tensors = tf.data.Dataset.from_tensor_slices([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
print(tensors)
for t in tensors:
    print(t)

<TensorSliceDataset shapes: (), types: tf.int32>
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(7, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(9, shape=(), dtype=int32)


In [7]:
# Automatic differentiation in gradient tape.
x = tf.ones((3, 3))
with tf.GradientTape() as t:
    t.watch(x)
    y = tf.reduce_sum(x)
    z = tf.multiply(y, y)

dz_dx = t.gradient(z, x)  # gradient of z wrt. x
print('dz_dx =', dz_dx)

# Notice GradientTape.gradient can only be called once on non-persistent tapes.
x = tf.ones((3, 3))
with tf.GradientTape() as t:
    t.watch(x)
    y = tf.reduce_sum(x)
    z = tf.multiply(y, y)

dz_dy = t.gradient(z, y)  # gradient of z wrt. y
print('dz_dy =', dz_dy)

print()
# In order to solve this inconvinience, we can create a persistent gradient tape and delete it after use.
x = tf.ones((3, 3))
with tf.GradientTape(persistent=True) as t:
    t.watch(x)
    y = tf.reduce_sum(x)
    z = tf.multiply(y, y)

dz_dx = t.gradient(z, x)
print('dz_dx =', dz_dx)
dz_dy = t.gradient(z, y)
print('dz_dy =', dz_dy)
del t

dz_dx = tf.Tensor(
[[18. 18. 18.]
 [18. 18. 18.]
 [18. 18. 18.]], shape=(3, 3), dtype=float32)
dz_dy = tf.Tensor(18.0, shape=(), dtype=float32)

dz_dx = tf.Tensor(
[[18. 18. 18.]
 [18. 18. 18.]
 [18. 18. 18.]], shape=(3, 3), dtype=float32)
dz_dy = tf.Tensor(18.0, shape=(), dtype=float32)


In [8]:
# A good thing of gradient tape is that they can handle statements like ;'if', 'while', etc.
def f(x, y):
    output = 1
    for i in range(y):
        if x > 1 and x < 5:
            output = tf.multiply(output, x)
    return output

x = tf.convert_to_tensor(2.0)
with tf.GradientTape() as t:
    t.watch(x)
    z = f(x, 5)
print(t.gradient(z, x))

tf.Tensor(80.0, shape=(), dtype=float32)


In [9]:
# Gradient tape can also deal with high order gradients.
x = tf.convert_to_tensor(1.0)  # Note that if eager execution is enabled, 'tf.Variable' is not valid.

with tf.GradientTape() as t:
    with tf.GradientTape() as t2:
        t2.watch(x)
        y = x * x * x
    dy_dx = t2.gradient(y, x)
    t.watch(x)
d2y_dx2 = t.gradient(dy_dx, x)

print('dy_dx =', dy_dx)
print('d2y_dx2 =', d2y_dx2)

dy_dx = tf.Tensor(3.0, shape=(), dtype=float32)
d2y_dx2 = tf.Tensor(6.0, shape=(), dtype=float32)
